In [1]:
import os 

import pandas as pd
import numpy as np


# import radial color histogram code, which can be found here:
# https://github.com/gmorinan/radialColorHistogram
from radialColorHistogram.color_density import ColorDensity
from radialColorHistogram.radial_split import RadialSplitter


from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import wordnet as wn
from collections import defaultdict
import regex as re
from nltk.tokenize import TweetTokenizer
from nltk.stem import SnowballStemmer

nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
nltk.download('stopwords')

try:
  import contractiond
except:
  !pip install contractions
  import contractions

tweet_tokenizer = TweetTokenizer()
# word_Lemmatized = WordNetLemmatizer()
stop = set(stopwords.words("english")) 
stemmer = SnowballStemmer(language="english")

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import FunctionTransformer

try:
  from zeugma.embeddings import EmbeddingTransformer
except:
  !pip install zeugma
  from zeugma.embeddings import EmbeddingTransformer


from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

try:
  from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter
except:
  !pip install textattack
  from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, EasyDataAugmenter

from sklearn.neighbors import KNeighborsClassifier

ModuleNotFoundError: ignored

In [ ]:

# import radial color histogram code, which can be found here:
# https://github.com/gmorinan/radialColorHistogram
from radialColorHistogram.color_density import ColorDensity
from radialColorHistogram.radial_split import RadialSplitter


### PP

In [ ]:
root = "/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/"
train_path = "train_images/train_images"
test_path = "test_images"

train_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/train.csv")
test_dt = pd.read_csv("/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/test.csv")
train_dt.head()

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0
3,4,448_image_batch_2.png,"""COME ON MAN, YOU KNOW THE THING.\r\nJUST ASK ...",troll,2
4,5,653_image_batch_2.png,"""Those who believe without reason cannot be co...",none,0


In [ ]:
def preprocess_txt(text):

  caps = True if text != text.lower() else False
  multiline = True if '\n' in text else False
  bullet = True if '- ' in text else False

  text = text.lower()   # ??
  text = re.sub(r"\n"," ",text)
  text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
  text = re.sub(r'(([^\s]+\.com)|([^\s]+\.net)|([^\s]+\.in))', '', text)
  text = re.sub(r'http\S+', '', text)
  
  # contractions fis
  expanded_words = []
  for word in text.split():
    expanded_words.append(contractions.fix(word))
  text = " ".join(expanded_words)

  #$# text = text.replace(r'\s+', ' ')
  text = text.replace("@","@ ")
  text = re.sub(r'[^a-zA-Z0-9 \@]', ' ', text) # ????
  ## text = re.sub('@[^\s]+','',text)

  # Single character removal
  text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)

  # Removing multiple spaces
  text = re.sub(r'\s+', ' ', text)

  #####text = tweet_tokenizer.tokenize(text)
  text = word_tokenize(text)
  text = [x for x in text if x not in stop]
  
  text = [stemmer.stem(x) for x in text]

  ##text = ["<CAPS>"] + text if caps else text
  text = ["<MULTILINE>"] + text if multiline else text
  ##text = ["<BULLET>"] + text if bullet else text


  text = " ".join(text)


  return text


def preprocess_txt_mini(text):

  text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', text)
  text = re.sub(r'(([^\s]+\.com)|([^\s]+\.net)|([^\s]+\.in))', '', text)
  text = re.sub(r'http\S+', '', text)

  text = re.sub(r'[^a-zA-Z0-9 \.,\?\\\'\"\:\;\#\%\!\&\-\@\n]', ' ', text) # ????

  return text


def tokenize(text): 
    tknzr = TweetTokenizer()
    return tknzr.tokenize(text)

def stem(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

en_stopwords = set(stopwords.words("english")) 

In [ ]:
def get_files_labels(subpath,csv):
    """Gets paths images within a folder, and their labels"""
    img_paths, img_labels, texts = [], [], []
    df = pd.read_csv(root+csv)
    i=0
    for index, row in df.iterrows():
        try:
          img, text, label = row["image id"], row["text"], int(row["label_num"])
        except:
          img, text, label = row["image id"], row["text"], -1
        # print(img_,": ",text_)
        
        ## Change this for img....>>>
        if not (i in [858, 1986, 1990] and csv=="train.csv"):
        #### if True:
          img_paths.append(root+subpath+'/'+img.strip())
          img_labels.append(label)
          ###texts.append(preprocess_txt(text))
          ###texts.append(text)
          texts.append(preprocess_txt_mini(text))
        i+=1
        #print('\r images: {} labels : {}'.format(len(img_paths), len(img_labels)) )
    return img_paths, img_labels, texts

In [ ]:
files_, y_, texts_ = get_files_labels(train_path,"train.csv")
files_test, y_test, texts_test = get_files_labels(test_path,"test.csv")


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    np.column_stack([files_, texts_]), y_, test_size=0.1, random_state=42, stratify=y_, shuffle=True)
X_test = np.column_stack([files_test,texts_test])
# X_train[0]

In [ ]:
##aug = EasyDataAugmenter()
##aug.augment(texts_[0])

In [ ]:
def newaug(sentences,aug = EmbeddingAugmenter(),easy=False ):
  new_sentences = []
  for sentence in sentences:
    if easy:
      sen = aug.augment(sentence)
      idx,le = 0,0
      for i in range(len(sen)):
        if len(sen[i]) > le:
          le = len(sen[i])
          idx = i
      new_sentences.append(sen[idx])     
    else:
      new_sentences.append(aug.augment(sentence)[0])
  return new_sentences

# newaug(X_train[:,1])[0]
# newaug(["HELLO from New York!","I'm fine!"],WordNetAugmenter())

In [ ]:
X_train_wn = newaug(X_train[:,1], aug=WordNetAugmenter())
print("Done0")
#X_train_emb = newaug(X_train[:,1], aug=EmbeddingAugmenter())
#print("Done1")
#X_train_ed = newaug(X_train[:,1], aug=EasyDataAugmenter(), easy=True)


Done0


In [ ]:

import pickle 

open_file = open("modpp__X_train_wn.pkl", "rb")
X_train_wn = pickle.load(open_file)
open_file.close()


In [ ]:
## print(X_train[0,1])
for i in range(X_train.shape[0]):
  X_train[i,1] = preprocess_txt(X_train[i,1])

for i in range(X_val.shape[0]):
  X_val[i,1] = preprocess_txt(X_val[i,1])

for i in range(X_test.shape[0]):
  X_test[i,1] = preprocess_txt(X_test[i,1])

In [ ]:
X_train_wn[0]

'When you finally drop that toxic girl in your life and you start burn  The Internet Scavengers'

In [ ]:
for i in range(X_train.shape[0]):
  X_train_wn[i] = preprocess_txt(X_train_wn[i])
  #X_train_emb[i] = preprocess_txt(X_train_emb[i])
  #X_train_ed[i] = preprocess_txt(X_train_ed[i])


In [ ]:
import pickle 

open_file = open("X_train_emb1.pkl", "rb")
X_train_emb = pickle.load(open_file)
open_file.close()


open_file = open("X_train_wn.pkl", "rb")
X_train_wn = pickle.load(open_file)
open_file.close()


In [ ]:
X_train_wn[0]

'final drop toxic girl life-tim start glow internet scaveng'

In [ ]:
def augment(sentences,n):
    import random
    random.seed(42)
    new_sentences = []
    for _ in range(n):
      for sentence in sentences:
        words = sentence.split()
        random.shuffle(words)
        new_sentences.append(' '.join(words))
    return new_sentences

In [ ]:
def get_synonyms_lexicon(path="ppdb-xl.txt"):
    synonyms_lexicon = {}
    text_entries = [l.strip() for l in open(path).readlines()]
    for e in text_entries:
        e = e.split(' ')
        k = e[0]
        v = e[1:len(e)]
        synonyms_lexicon[k] = v
    return synonyms_lexicon

synonyms = get_synonyms_lexicon()

In [ ]:
X_train2 = augment(X_train[:,1],1)
X_train[:,1][0],X_train2[0]

('final drop toxic girl life start glow internet scaveng',
 'girl glow internet life scaveng toxic start final drop')

In [ ]:
X_train1 = []
for sentence in X_train[:,1]:
  new_sentence = []
  for word in sentence.split():
    if word in synonyms.keys():
      new_sentence.append(synonyms[word][0])
    else:
      new_sentence.append(word)
  X_train1.append(" ".join(new_sentence))

X_train1[0]

'final decline toxic girl living start glow web scaveng'

In [ ]:
def img_col(X):
    return X[:,0]

def text_col(X):
    return X[:,1]


In [ ]:
preprocess_txt("ali imgflip.com")

'ali'

In [ ]:
tweet_tokenizer.tokenize(text)

NameError: ignored

In [ ]:
X_train_______ = X_train.copy()
X_train1_______ = X_train1.copy()

In [ ]:
for i in range(X_train.shape[0]):
  if X_train[i,1] != X_train_______[i,1]:
    print(i, '\n', X_train[i,1],'\n', X_train_______[i,1])
    break


In [ ]:
for i in range(X_train.shape[0]):
  if  X_train1[i] != X_train1_______[i]:
    print(i, '\n', X_train1[i],'\n', X_train1_______[i])
    break


###  Image

In [ ]:
img_pipe = Pipeline([
                     ("cd",ColorDensity(splitter=RadialSplitter(nrings=1, nqslices=1), 
                                        color_model='HSV', n_bins=3, 
                                        scaler=True, log_transform=True)),
                     #("svc", SVC(kernel='linear',probability=True, C=1, max_iter=50000, random_state=2020))
                     ('clf', OneVsRestClassifier( LogisticRegression(class_weight="balanced", random_state=42) ))
                    ],
                    verbose = 1)

img_pipe.fit(X_train[:,0],y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


[Pipeline] ................ (step 1 of 2) Processing cd, total=10.2min
[Pipeline] ............... (step 2 of 2) Processing clf, total=   0.3s


Pipeline(memory=None,
         steps=[('cd',
                 <radialColorHistogram.color_density.ColorDensity object at 0x7f9a25976190>),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=1.0,
                                                                  class_weight='balanced',
                                                                  dual=False,
                                                                  fit_intercept=True,
                                                                  intercept_scaling=1,
                                                                  l1_ratio=None,
                                                                  max_iter=100,
                                                                  multi_class='auto',
                                                                  n_jobs=None,
                                                                  penalty='l2',
                

In [ ]:
# make predictions
#y_pred_train_img = img_pipe.predict(X_train[:,0])
y_pred_val_img = img_pipe.predict(X_val[:,0])
y_pred_test_img = img_pipe.predict(X_test[:,0])

In [ ]:
from sklearn.metrics import classification_report
# print(classification_report(y_train, y_pred_train_img))
print(classification_report(y_val, y_pred_val_img))

              precision    recall  f1-score   support

           0       0.24      0.28      0.26        60
           1       0.39      0.38      0.39        69
           2       0.35      0.31      0.33        70

    accuracy                           0.33       199
   macro avg       0.33      0.32      0.33       199
weighted avg       0.33      0.33      0.33       199



In [ ]:
print(*y_pred_test_img,sep="\n")

In [ ]:
fimg_v = img_pipe.predict_proba(X_val[:,0])
fimg_t = img_pipe.predict_proba(X_test[:,0])

In [ ]:
import pickle
filename = 'img_pipe.sav'
pickle.dump(img_pipe, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
img_pipe.predict_proba(X_val[:5,0]) == loaded_model.predict_proba(X_val[:5,0])

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

### Text

In [ ]:
from sklearn.preprocessing import FunctionTransformer

"""
text_pipe = Pipeline([
                     ("tdidf",TfidfVectorizer(
                        min_df=4, norm='l2', 
                        encoding='utf-8', 
                        ngram_range=(1,   ))),
                     ("svc",SVC(kernel='linear',probability=True, C=10, max_iter=50000, random_state=2020))
                    ],
                    verbose = 1)
"""


text_pipe = Pipeline([
        #('vect', CountVectorizer()),
        #('tfidf', TfidfTransformer()),
        ("tdidf",TfidfVectorizer(
                        min_df=3, norm='l2', 
                        encoding='utf-8', 
                        lowercase = True,
                        ngram_range=(1, 2),
                        preprocessor = None)),
        ('clf', SVC(C=2,random_state=42,probability=True))
        ],
        verbose = 1)

######################################

'''
## BEST
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = None,
                  min_df=3, max_df=600,
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', LinearSVC(max_iter=5000, penalty="l2", loss='squared_hinge', multi_class='ovr', random_state=42)) ],
        verbose = 1)
'''


'''
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 3),
                  preprocessor = None,
                  min_df=9, 
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        ('clf', XGBClassifier(
    gamma=4,
    eta=0.8,
    reg_alpha=0.5,
    reg_lambda=0.5
) )],
        verbose = 1)
'''

'''
### emb = EmbeddingTransformer('fasttext')

text_pipe = Pipeline([
        # ('pp', FunctionTransformer(preprocess_txt) ),
        ('vect', EmbeddingTransformer('glove') ),  # glove, word2vec, fasttext
        ('clf', SVC(C=2, random_state=42) )],
        verbose = 1)
'''

'''
text_pipe = Pipeline([
        # ('pp', FunctionTransformer(preprocess_txt) ),
        ('vect', EmbeddingTransformer('glove') ),  # glove, word2vec, fasttext
        # ('clf', LogisticRegression(class_weight="balanced", random_state=42),
        ('clf', OneVsRestClassifier( LogisticRegression(solver="sag", class_weight="balanced", random_state=42) ) )
        ],

        verbose = 1)
'''

'''
text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = None,
                  min_df=3, max_df=600,
                   ) ),
        ('tfidf', TfidfTransformer(sublinear_tf=False)),
        # ('clf', LogisticRegression(class_weight="balanced", random_state=42),
        ('clf', OneVsRestClassifier( LogisticRegression(class_weight="balanced", random_state=42) ) 
        )],
        verbose = 1)

'''


text_pipe.fit(list(X_train[:,1])+X_train_wn+X_train1,3*[x for x in y_train])

[Pipeline] ............. (step 1 of 2) Processing tdidf, total=   0.2s
[Pipeline] ............... (step 2 of 2) Processing clf, total=  24.5s


Pipeline(memory=None,
         steps=[('tdidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=3, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 SVC(C=2, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degre

In [ ]:
# make predictions
y_pred_train_text = text_pipe.predict(X_train[:,1])
y_pred_val_text = text_pipe.predict(X_val[:,1])
y_pred_test_text = text_pipe.predict(X_test[:,1])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report([x for x in y_train ], y_pred_train_text))
print(classification_report([x for x in y_val], y_pred_val_text))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       543
           1       1.00      1.00      1.00       618
           2       0.99      1.00      0.99       628

    accuracy                           0.99      1789
   macro avg       0.99      0.99      0.99      1789
weighted avg       0.99      0.99      0.99      1789

              precision    recall  f1-score   support

           0       0.40      0.32      0.36        60
           1       0.41      0.45      0.43        69
           2       0.42      0.46      0.44        70

    accuracy                           0.41       199
   macro avg       0.41      0.41      0.41       199
weighted avg       0.41      0.41      0.41       199



In [ ]:
print(*y_pred_test_text,sep="\n")

In [ ]:
ftxt_v = text_pipe.predict_proba(X_val[:,1])
ftxt_t = text_pipe.predict_proba(X_test[:,1])

In [ ]:
import pickle

open_file = open("lstm__val_pred_2.pkl", "rb")
flstm_v = pickle.load(open_file)
open_file.close()

open_file = open("lstm__test_pred_2.pkl", "rb")
flstm_t = pickle.load(open_file)
open_file.close()

In [ ]:
f = np.argmax(ftxt_v+fimg_v+flstm_v ,axis=1)
f.shape

(199,)

In [ ]:
print(classification_report([x for x in y_val], f))

              precision    recall  f1-score   support

           0       0.42      0.32      0.36        60
           1       0.49      0.54      0.51        69
           2       0.42      0.47      0.44        70

    accuracy                           0.45       199
   macro avg       0.44      0.44      0.44       199
weighted avg       0.45      0.45      0.44       199



In [ ]:
##SUB12

ft = np.argmax(ftxt_t+fimg_t+flstm_t ,axis=1)
print(*ft,sep='\n')

1
2
2
2
2
2
2
0
1
1
2
2
2
2
2
2
1
2
2
2
2
2
2
0
1
2
1
0
1
0
2
2
2
1
0
2
1
1
2
1
2
2
1
0
0
1
0
2
2
0
1
2
2
0
0
1
0
2
1
0
0
2
2
1
0
2
0
2
0
2
2
2
2
2
1
1
2
0
2
2
0
2
2
1
2
1
0
2
1
2
2
2
0
1
2
1
2
2
2
2
0
2
1
0
2
1
2
2
1
2
0
0
0
0
0
2
1
2
1
1
2
1
0
0
2
2
2
2
2
1
2
2
2
1
1
2
2
1
2
1
2
2
1
2
2
0
2
0
1
2
2
2
2
2
1
2
0
1
0
1
1
1
1
2
0
2
0
2
2
1
1
1
1
2
2
2
1
2
2
1
2
0
1
2
1
0
1
2
1
2
0
2
2
2
0
2
1
2
0
2
1
0
1
2
0
2
1
1
1
0
1
1
2
0
0
1
2
1
2
2
1
0
1
1
1
1
0
1
1
2
2
1
1
2
1
1
1
2
0
0
0
0
1
2
2
0
1
2
1
2
2
1
1
2
2
2
0
0
1
1
1
1
1
2
2
2
0
0
1
1
1
1
2
2
0
2
2
1
0
2
0
2
1
0
0
1
1
2
1
1
0
1
1
2
2
0
2
0
0
0
1
0
2
1
2
1
0
0
2
0
1
0
2
2
1
0
1
2
0
1
1
2
0
2
0
2
1
0
0
0
2
1
0
0
1
1
2
2
1
0
2
1
1
2
0
1
1
0
0
2
2
0
0
1
2
1
1
1
2
1
0
0
2
2
0
1
1
0
2
2
2
2
2
2
0
0
1
2
2
0
1
0
0
1
0
0
2
2
1
0
0
2
1
1
2
2
1
0
2
0
1
2
2
0
1
2
2
1
2
1
2
2
1
1
2
0
0
1
0
0
1
1
2
2
2
0
1
1
2
0
1
2
0
2
1
0
2
2
1
1
2
2
2
0
1
1
1
0
0
1
2
2
2
1
0
0
1
1
0
0
1
1
1
2
0
2
0
2
2
2
0
2
2
0
2
2
0
2
1
1
2
2
1
0
1
2
2
2
2
1
1
2
2
0
1
1
0
1
0
2


#### Pipeline Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV


tuned_parameters = [{'svc__kernel': ['rbf'], 'svc__gamma': [1e-3, 1e-4],
                     'svc__C': [1, 10, 100, 1000]},
                    {'svc__kernel': ['linear'], 'svc__C': [1, 10, 100, 1000]}]


text_pipe = Pipeline([
        ('vect', CountVectorizer(analyzer = 'word',
                  lowercase = True,
                  ngram_range=(1, 2),
                  preprocessor = preprocess_txt,) ),
        ('tfidf', TfidfTransformer()),
        ('svc', SVC(random_state=42)) ],
        verbose = 1)



clf = GridSearchCV(
    text_pipe, tuned_parameters, scoring='f1_macro',
    refit=True,verbose=3
)

clf.fit(X_train[:,1],y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173, total=   1.4s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173, total=   1.4s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.174, total=   1.4s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173, total=   1.3s
[CV] svc__C=1, svc__gamma=0.001, svc__kernel=rbf .....................
[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.7s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.4s
[CV]  svc__C=1, svc__gamma=0.001, svc__kernel=rbf, score=0.173,

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.2min finished


[Pipeline] .............. (step 1 of 3) Processing vect, total=   0.8s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing svc, total=   0.6s
Best parameters set found on development set:

{'svc__C': 1000, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}

Grid scores on development set:

0.173 (+/-0.001) for {'svc__C': 1, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 1, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 10, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 10, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.198 (+/-0.017) for {'svc__C': 100, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.173 (+/-0.001) for {'svc__C': 100, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.395 (+/-0.049) for {'svc__C': 1000, 'svc__gamma': 0.001, 'svc__kernel': 'rbf'}
0.198 (+/-0.017) for {'svc__C': 1000, 'svc__gamma': 0.0001, 'svc__kernel': 'rbf'}
0.392 

In [ ]:
# make predictions
y_pred_train_text = clf.predict(X_train[:,1])
y_pred_val_text = clf.predict(X_val[:,1])
y_pred_test_text = clf.predict(X_test[:,1])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train_text))
print(classification_report(y_val, y_pred_val_text))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       513
           1       1.00      1.00      1.00       586
           2       1.00      0.99      1.00       593

    accuracy                           1.00      1692
   macro avg       1.00      1.00      1.00      1692
weighted avg       1.00      1.00      1.00      1692

              precision    recall  f1-score   support

           0       0.30      0.26      0.28        91
           1       0.43      0.49      0.46       103
           2       0.39      0.39      0.39       105

    accuracy                           0.38       299
   macro avg       0.38      0.38      0.38       299
weighted avg       0.38      0.38      0.38       299



In [ ]:
## text_pipe.predict_proba(X_val[:,1])

In [ ]:
y_pred_test

In [ ]:
print(*y_pred_test_text,sep="\n")

### Combine

In [ ]:
y_pred_train_comb = np.column_stack([y_pred_train_img,y_pred_train_text])
y_pred_val_comb = np.column_stack([y_pred_val_img,y_pred_val_text])
#y_pred_test_comb = np.column_stack([y_pred_test_img,y_pred_test_text])


In [ ]:
#clf = DecisionTreeClassifier()
#clf = RandomForestClassifier()
clf = SVC(random_state=42)
clf = clf.fit(y_pred_train_comb, y_train)


In [ ]:
# make predictions
y_pred_train_comb__ = clf.predict(y_pred_train_comb)
y_pred_val_comb__ = clf.predict(y_pred_val_comb)
#y_pred_test_comb__ = clf.predict(y_pred_test_comb)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train_comb__))
print(classification_report(y_val, y_pred_val_comb__))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       543
           1       0.88      0.89      0.89       618
           2       0.86      0.89      0.87       628

    accuracy                           0.88      1789
   macro avg       0.88      0.88      0.88      1789
weighted avg       0.88      0.88      0.88      1789

              precision    recall  f1-score   support

           0       0.38      0.33      0.36        60
           1       0.48      0.52      0.50        69
           2       0.40      0.41      0.41        70

    accuracy                           0.43       199
   macro avg       0.42      0.42      0.42       199
weighted avg       0.42      0.43      0.42       199



In [ ]:
## SUB8
print(*y_pred_test_comb__,sep="\n")

2
2
2
2
2
2
1
0
1
1
2
2
2
2
2
2
2
2
0
1
1
2
0
0
2
1
1
0
2
0
0
0
2
2
0
2
2
0
1
2
0
2
1
1
2
1
0
1
2
1
2
2
2
0
2
2
0
2
0
1
1
0
2
1
2
1
2
1
1
2
2
2
2
2
0
2
1
0
1
2
1
1
2
1
2
1
2
2
1
2
2
2
0
1
2
1
1
1
1
2
1
2
1
2
2
2
2
2
2
2
2
0
2
2
0
0
1
0
1
0
1
1
2
2
2
2
2
2
2
2
1
2
2
1
2
2
1
0
2
0
2
0
2
2
2
2
2
2
1
1
1
2
2
1
2
2
0
1
0
1
1
1
1
1
2
2
0
1
2
1
1
1
1
1
2
2
2
2
1
2
1
2
0
1
1
1
1
2
2
2
0
2
1
2
2
1
0
0
2
2
1
2
1
2
0
2
1
2
1
2
1
0
2
2
1
1
2
1
2
0
2
2
1
1
1
1
0
1
1
0
1
1
1
0
1
1
1
2
1
2
1
0
1
2
1
0
1
2
1
2
2
2
2
2
2
2
0
2
1
1
1
1
1
2
0
0
1
1
1
1
2
2
2
1
1
2
0
2
2
2
0
1
1
1
0
2
2
2
1
2
0
1
2
2
2
0
1
2
0
2
1
0
2
1
1
2
2
0
1
2
1
0
1
2
2
2
1
1
0
1
2
2
1
1
2
1
1
0
1
2
2
1
2
1
0
1
2
2
1
2
2
1
1
2
0
2
0
1
2
2
2
0
1
1
2
2
1
2
1
1
1
0
2
2
0
0
1
2
2
2
0
2
2
2
0
2
2
2
2
2
2
2
2
1
1
2
2
1
1
1
1
2
1
1
2
0
1
2
0
0
1
0
0
0
2
2
2
1
2
1
1
2
2
2
2
1
1
0
0
2
1
0
2
1
1
0
2
0
2
2
1
1
1
0
0
0
2
1
2
1
2
2
2
2
0
1
1
0
1
2
1
1
2
1
2
0
2
1
0
0
1
1
0
2
1
2
2
2
0
2
0
1
1
1
1
1
0
0
0
0
1
2
2
2
2
1
2
2
1
0
1
1
0
1
2
2
2
2
0
2


## Old

In [ ]:
# define our settings for processing the images 
rs = RadialSplitter(nrings=1, nqslices=1)
cd = ColorDensity(splitter=rs, color_model='HSV', n_bins=3, 
                  scaler=True, log_transform=True)

In [ ]:
# process images into radial color histogram data
X_train = cd.fit_transform(files_train)
print("done")
X_val = cd.transform(files_val)
X_test = cd.transform(files_test)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


done


In [ ]:
"""
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC


tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

clf = GridSearchCV(
    SVC(), tuned_parameters, scoring='f1_macro',
    refit=True,verbose=3
)
clf.fit(X, y)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()
"""

In [ ]:
# fit a linear Support Vector Machine classifier
svc = LinearSVC(C=1, max_iter=50000)
svc.fit(X_train, y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=50000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
# make predictions
y_pred_train = svc.predict(X_train)
y_pred_val = svc.predict(X_val)
y_pred_test = svc.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_train, y_pred_train))
print(classification_report(y_val, y_pred_val))

              precision    recall  f1-score   support

           0       0.47      0.38      0.42       512
           1       0.45      0.49      0.47       584
           2       0.48      0.51      0.50       593

    accuracy                           0.47      1689
   macro avg       0.47      0.46      0.46      1689
weighted avg       0.47      0.47      0.46      1689

              precision    recall  f1-score   support

           0       0.32      0.31      0.31        91
           1       0.35      0.37      0.36       103
           2       0.33      0.32      0.33       105

    accuracy                           0.33       299
   macro avg       0.33      0.33      0.33       299
weighted avg       0.33      0.33      0.33       299



In [ ]:
print(*y_pred_test,sep="\n")

2
2
2
2
2
0
0
1
1
2
2
1
1
2
1
1
2
0
0
1
0
2
2
0
0
1
2
1
1
1
2
1
1
0
2
2
0
1
1
0
2
1
0
2
1
2
2
0
1
1
1
2
2
1
1
0
0
2
2
0
0
2
0
2
0
2
2
1
2
1
1
1
1
2
1
0
0
2
0
2
2
2
2
2
0
2
0
2
1
2
1
0
0
2
0
1
1
1
0
2
0
2
1
0
1
1
0
1
0
1
2
1
1
0
1
2
2
2
0
2
0
1
1
0
1
2
2
1
1
0
2
1
0
2
2
2
2
0
2
2
1
0
1
1
1
2
2
0
2
1
1
0
1
2
1
1
2
1
1
2
1
1
1
2
1
1
2
2
2
2
0
2
2
2
0
2
2
2
2
0
2
2
1
2
1
1
1
0
1
1
2
0
1
0
1
2
1
2
0
1
2
1
1
1
2
2
0
0
2
0
0
1
2
1
1
0
1
0
2
2
0
2
2
2
2
0
2
2
1
1
2
2
0
2
1
1
0
1
0
2
1
1
1
0
1
2
1
2
2
2
1
0
0
1
1
2
0
2
1
1
2
2
2
0
1
2
0
1
1
2
0
1
2
0
2
0
0
1
1
1
0
1
2
1
0
1
0
1
0
2
2
0
1
2
2
0
0
0
1
2
1
2
2
0
2
0
2
2
2
1
1
2
1
2
1
1
2
2
0
1
0
1
1
1
0
1
2
0
0
0
1
0
2
1
1
2
1
0
2
0
2
2
1
2
1
2
1
2
0
2
1
2
1
1
0
2
1
1
2
2
2
2
0
1
1
2
0
1
2
2
2
0
1
2
2
1
2
0
2
0
2
0
0
2
1
2
2
2
2
0
0
2
2
2
2
1
0
1
0
1
0
2
2
2
1
2
0
2
1
2
1
0
0
1
0
0
2
1
2
2
2
1
1
2
0
1
0
1
2
1
1
1
2
2
1
2
2
2
1
2
0
2
2
1
2
0
2
0
1
0
0
2
1
1
0
0
2
1
2
1
2
2
2
0
1
0
2
2
0
2
2
2
1
2
2
2
0
2
2
1
2
2
0
0
1
1
2
1
1
0
1
2
1
2
2
1
2
1
0
1


In [ ]:
import os

file_names_list = files_test

if all(list(map(os.path.isfile,file_names_list))):
  print("t")
else:
  print("g")

g


In [ ]:
from cv2 import imread, cvtColor, calcHist, COLOR_BGR2RGB
import cv2
for i in range(len(files_train[:])):
  try:
    image = imread(files_train[0+i])  # read image
    image = cvtColor(image, COLOR_BGR2RGB) 
    print(0+i,end= " ")
  except:
    print("\n >>> ",0+i)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
len(X[1])

108

In [ ]:
for x in X:
  assert len(x) == 108

In [ ]:
type(X)

numpy.ndarray

In [ ]:
svc = LinearSVC(C=1, max_iter=50000)
svc.fit(X, y)
y_pred_ = svc.predict(X)
#y_pred_val = svc.predict(X_val)
y_pred_test = svc.predict(X_test)

print(classification_report(y, y_pred_))

              precision    recall  f1-score   support

           0       0.48      0.35      0.40       526
           1       0.48      0.55      0.51       572
           2       0.49      0.54      0.51       591

    accuracy                           0.48      1689
   macro avg       0.48      0.48      0.48      1689
weighted avg       0.48      0.48      0.48      1689



In [ ]:
print(*y_pred_test,sep="\n")

2
2
2
2
2
0
0
1
1
2
2
1
1
2
1
1
2
2
0
1
0
2
2
1
0
1
2
1
1
1
1
1
1
0
2
2
0
1
2
0
2
2
0
2
2
2
2
2
1
1
1
2
2
2
1
1
1
2
2
0
0
2
0
2
1
2
2
1
2
1
1
2
1
0
1
0
2
2
0
2
2
2
2
2
0
2
0
2
1
1
1
0
0
2
0
1
1
1
0
2
0
2
1
0
1
1
0
1
0
1
0
1
1
0
1
2
2
2
0
1
0
1
0
0
1
1
2
1
1
0
2
1
0
2
2
2
0
0
2
2
1
0
1
1
1
2
0
0
2
1
1
1
1
2
1
1
2
1
1
2
1
1
1
2
1
1
2
2
2
2
0
2
2
0
0
2
2
2
2
1
2
2
1
2
1
1
1
0
1
1
2
0
1
2
2
2
2
2
0
1
2
1
1
1
2
2
0
0
2
1
0
1
2
1
1
2
1
0
2
2
0
2
2
2
2
0
2
1
1
1
2
2
0
2
1
1
0
1
2
2
1
1
0
0
1
2
1
1
2
2
1
2
0
1
1
2
0
2
1
1
2
2
1
0
1
0
0
1
1
2
0
2
2
0
0
0
0
2
1
1
0
2
2
1
0
1
0
1
0
2
2
0
1
0
2
2
1
0
1
2
1
2
2
0
2
0
1
0
2
1
1
2
1
2
1
1
2
2
1
2
0
2
1
1
0
2
2
0
0
1
1
0
2
1
1
2
1
2
2
1
1
2
1
2
0
2
1
2
0
2
1
2
1
1
0
2
1
1
2
2
2
2
0
1
1
2
0
1
2
1
2
0
1
2
2
1
1
1
1
1
2
0
1
2
1
2
2
2
2
1
0
2
1
2
2
1
0
1
0
1
0
2
2
2
1
2
0
2
1
2
1
0
0
1
0
0
1
1
2
2
2
1
1
2
0
0
0
1
2
1
1
1
2
1
1
1
0
2
0
2
0
2
2
1
1
0
2
0
1
0
0
2
1
0
0
0
2
1
2
1
2
2
2
0
0
0
2
2
0
2
0
2
1
2
2
2
0
1
2
1
2
2
1
0
1
1
2
2
1
0
1
2
1
1
2
1
0
1
1
1


In [ ]:
stop

In [ ]:
X_val[0]

array(['/content/drive/MyDrive/MTL782-Memes Classifier/dataminingmtl782/train_images/train_images/image_4723.jpg',
       'harri potter movi cast hollywood star remus lupin would play greg kinnear'],
      dtype='<U545')

In [ ]:
import pickle

open_file = open("modpp__X_train_wn.pkl", "wb")
pickle.dump(X_train_wn, open_file)
open_file.close()



In [ ]:
iii=3
X_train_wn[iii], X_train_emb[iii]

('mather cri franchis domino pizza make offer refus 03 16 parmesan lettuc burn 1 2 medium 2 top pizza 5 99 godfath trilog godfath crono 2 p ort rount corpor alagam',
 'mather call franchis domino pizza make offer refus 03 16 parmesan bread bite 1 2-medium 2-top pizza onto 5 99 godfath triad godfath crono 2 p ort rount firm alagam')